**Imports and Setup**

In [1]:
import os
import dotenv
import pandas as pd
import duckdb

# setup
API_KEY = dotenv.get_key("./.env", "api_min")

# API root entry point
URL = "https://api.mindat.org"

# authorization header that must be included with each request.
headers = {'Authorization': 'Token ' + API_KEY}

# directories
DATA_DIR = os.path.join(os.path.abspath('./'), 'data')
QUERIES_DIR = os.path.join(os.path.abspath('./'), 'queries')

# fixing pandas analyzer
duckdb.execute("SET GLOBAL pandas_analyze_sample = 100_000")

In [2]:
silver_geomaterials = duckdb.read_csv(os.path.join(DATA_DIR,'silver_geomaterials.csv'))
silver_geomaterials

┌───────────────┬────────────┬──────────────────────┬───┬──────────────────┬─────────┬─────────┬─────────────┐
│ idGeomaterial │  idLongId  │        idGUID        │ … │ descShortcodeIma │ vlRiMin │ vlRiMax │ vlWeighting │
│     int64     │  varchar   │       varchar        │   │     varchar      │ double  │ double  │    int64    │
├───────────────┼────────────┼──────────────────────┼───┼──────────────────┼─────────┼─────────┼─────────────┤
│          7730 │ 1:1:7730:5 │ 0d34f6ec-8a30-42a2…  │ … │ NULL             │    NULL │    NULL │           0 │
│          7731 │ 1:1:7731:2 │ 77c795d6-44f3-4ebc…  │ … │ NULL             │    NULL │    NULL │           0 │
│          7732 │ 1:1:7732:9 │ 652c7d68-5865-45a2…  │ … │ NULL             │    NULL │    NULL │           0 │
│          7734 │ 1:1:7734:3 │ e2e040a9-e8f9-47ca…  │ … │ NULL             │    NULL │    NULL │           0 │
│          7735 │ 1:1:7735:0 │ a6619f65-3944-4666…  │ … │ NULL             │    NULL │    NULL │           1 │
│

In [18]:
query_nb = """

    -- meta data minerals
    select
        idGeomaterial,
        idGroup,
        descName,
        descNameCorrected,
        rtrim(ltrim(descElements, '['), ']') as descElements,
        descEntryTypeText,
        vlDiscoveryYear,
        dtUpdatedTime,
        vlWeighting,
        

        descDiapheny,
        descCleavage,
        descParting,
        descTenacity,
        descColour,
        descLustreType,
        descStreak,
        descDescriptionShort

    from silver_geomaterials
    where descEntryTypeText = 'mineral'
    order by descName

"""
duckdb.sql(query_nb).show(max_width=10000, max_rows=20)

┌───────────────┬─────────┬──────────────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────────┬───────────────────┬─────────────────┬─────────────────────┬─────────────┬─────────────────────────┬────────────────────────────────────┬─────────────┬──────────────┬────────────────────────────────────────────────────────────────────┬───────────────────────────┬─────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│ idGeomaterial │ idGroup │                   descName                   │              descNameCorrected               │             descElements             │ descEntryTypeText │ vlDiscoveryYear │    dtUpdatedTime    │ vlWeighting │      descDiapheny       │            descCleavage            │ descPa

In [16]:
query_nb = """

select
	*
from silver_geomaterials
where descEntryTypeText = 'mineral' 
--group by 1 
--order by 2 desc
limit 100


"""

duckdb.sql(query_nb).show(max_width=10000, max_rows=20)

┌───────────────┬────────────┬──────────────────────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────┬────────────────┬───────────────┬───────────────────┬────────────┬───────┬──────────────┬─────────┬─────────────┬───────────────────┬──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────┬─────────────────────────────────────┬──────────────────────────────┬────────────┬─────────┬────────────────┬─────────────────────┬────────────────┬─────────────────┬──────────────┬──────────────┬─────────────┬──────────────┬──────────────┬──────────────┬─────────────────────

In [4]:
with open(os.path.join(QUERIES_DIR, 'silver_minerals.sql')) as f: 
    silver_geomaterials = duckdb.read_csv(os.path.join(DATA_DIR, 'silver_geomaterials.csv'),sample_size=-1)
    q = f.read()
    f.close()
    duckdb.sql(q).show(max_width=10000)

┌───────────────┬──────────────┬──────────────────────────────────────┬──────────────────────────────────────────────┬──────────────────────────────────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────────────┬──────────────┬───────────────┬────────────┬──────────────────────┬─────────────┐
│ idGeomaterial │   idLongId   │                idGUID                │                   descName                   │              descNameCorrected               │                                                                                                                      descDescriptionShort                                                                                                                       │   